# Funciones 

Este NOTEBOOK nos ayudará a poder trabajar en el ETL y no estar construyendo las mismas funciones repetitivas.

### IMPORTAMOS LIBRERÍAS NECESARIAS

In [7]:
import pandas as pd
import numpy as np
import re

### Funcion de CARGA de los dataset

Carga de los archvios de excel y devuelve un diccionario de DATAFRAME

En resumen, esta función simplifica la carga de datos desde un archivo Excel en uno o más DataFrames de Pandas, facilitando el manejo de múltiples hojas en un solo llamado a la función.

    Parameters:
    - archivo (str): Ruta del archivo Excel.
    - hojas (list): Lista de nombres de hojas a cargar.
    - engine (str, optional): Motor de Excel a utilizar. Por defecto, 'openpyxl'.

    Returns:
    dfs: Un diccionario donde las claves son los nombres de las hojas y los valores son DataFrames correspondientes.

    Example:
    >>> datos = cargar_datos_desde_excel('archivo.xlsx', ['Hoja1', 'Hoja2'])
    >>> df_hoja1 = datos['Hoja1']
    >>> df_hoja2 = datos['Hoja2']


In [8]:
def carga_datos_excel(archivo, hojas, engine='openpyxl'):
    xls_file = pd.ExcelFile(archivo, engine=engine)
    dfs = {}

    for hoja in hojas:
       df = pd.read_excel(xls_file, hoja)
       dfs[hoja] = df

    return dfs

### Funcion de ANALISIS de los VALORES

En resumen, esta función proporciona un análisis de las columnas de un DataFrame en busca de la presencia del valor 'SD', proporcionando información sobre la cantidad y el porcentaje de ocurrencias en cada columna.

In [9]:
def analizar_valores_sd(dataframe):

    columnas_con_sd = dataframe.columns # Obtiene la lista de nombres de columnas del DataFrame y la almacena en la variable
    resultados = []

    for columna in columnas_con_sd: # Itera a través de cada columna en columnas_con_sd.
        cantidad_sd = dataframe[columna].eq('SD').sum() # Para cada columna, cuenta la cantidad de veces que aparece el valor 'SD'
        porcentaje_sd = (cantidad_sd / len(dataframe)) * 100 # calcula el porcentaje de 'SD' en esa columna en relación con el tamaño total del DataFrame.
        resultados.append({'Columna': columna, 'Cantidad de SD': cantidad_sd, 'Porcentaje de SD': porcentaje_sd}) # La información se almacena en una lista de diccionarios llamada resultados.

    resultados_df = pd.DataFrame(resultados) #Convierte la lista de diccionarios resultados en un DataFrame de Pandas.
    resultados_con_sd = resultados_df[resultados_df['Cantidad de SD'] > 0] #Filtra el DataFrame para incluir solo las filas donde la 'Cantidad de SD' es mayor que cero

    return resultados_con_sd #La función devuelve el DataFrame resultante, que contiene información sobre las columnas que tienen al menos un valor 'SD', incluyendo la cantidad y el porcentaje de ocurrencias de 'SD' en cada una.

### Función para extraer las coordenadas CABA

- Esta función toma un argumento texto, que es el texto de entrada del cual se intentarán extraer las coordenadas
- Utiliza una expresión regular (re.findall) para buscar todas las coincidencias de números decimales (que tienen un punto como separador decimal) en el texto. Estas coincidencias se almacenan en la lista coordenadas.
- return float(coordenadas[0]), float(coordenadas[1]): Si se encontraron dos coordenadas, se convierten en números de punto flotante (float) y se devuelven como una tupla de dos valores.
- Luego, se aplica esta función a la columna 'XY (CABA)' de df_siniestros utilizando el método .map(). Esto se hace para cada fila de la columna, y los resultados se almacenan en dos nuevas columnas, 'coordenada_x' y 'coordenada_y', utilizando zip(*...).


In [10]:
def extraer_coordenadas(texto):
    # Utilizamos una expresión regular para extraer las coordenadas
    coordenadas = re.findall(r'\d+\.\d+', texto)
    if len(coordenadas) == 2:
        return float(coordenadas[0]), float(coordenadas[1])
    else:
        return None, None